# Kryptografia asymetryczna - kryptosystem RSA 
Kryptografia asymetryczna charakteryzuje się wykorzystaniem **pary kluczy publiczny-prywatny** (stąd nazwa kryptografia z kluczem publicznym). Klucz publiczny może być swobodnie dystrybuowany otwartym kanałem i służy do szyfrowania (a także do weryfikowania podpisu). Klucz prywatny musi być utrzymywany w tajności i służy do deszyfrowania (lub tworzenia podpisu). 

Chronologicznie pierwszym kryptosystemem asymetrycznym był protokół wymiany kluczu Diffiego-Hellmana-Merkla. Służy on bezpiecznej wymiany danych, które mogą być wykorzystane jako tajne klucze kryptograficzne lub mogą być użyte do wyprodukowania kluczy. 

Najbardziej znanym kryptosystem z kluczem publicznym jest RSA (nazwa pochodzi od wynalazów: Rivest, Shamir i Adlemann). RSA umożliwia szyfrowanie danych jak również realizację podpisu cyfrowego. Bezpieczeństwo RSA opiera się na obliczeniowej trudności rozwiązania **problemu faktoryzacji liczb całkowitych złożonych**. 

In [1]:
## Funkcje pomocnicze 
def gcd(a, b):
    # GCD - Greatest Common Divisor, Największy wspólny dzielnik 
    while a != 0:
        a, b = b % a, a
    return b

def findModInverse(a, m):
    # Zwraca liczbę x odwrotną do a ciele skończonym modulo m 
    # czyli (a*x) % m =1 

    if gcd(a, m) != 1:
        return None #a i m muszą być względnie pierwsze aby istniał element odwrotny 

    # Rozszerzony algorytm Euklidesa 
    u1, u2, u3 = 1, 0, a
    v1, v2, v3 = 0, 1, m
    while v3 != 0:
        q = u3 // v3 # // operator dzielenie całkowitoliczbowego 
        v1, v2, v3, u1, u2, u3 = (u1 - q * v1), (u2 - q * v2), (u3 - q * v3), v1, v2, v3
    return u1 % m

## Generowanie kluczy w kryptosystemie RSA

### 1. Losujemy dwie duże liczby pierwsze 
Potrzebujemy dwóch liczb pierwszych o naprawdę dużych rozmiarach - 2048 bitów obecnie uważa się niezbyt bezpieczny wybór. 4096 bitów jest z kolei wielkością nieco kłopotliwą w użytkowaniu. 
#### Skąd wziąć liczbę pierwszą? 
**Wylosować i sprawdzić czy jest pierwsza!**


Test probabilistyczny, np. Rabina-Millera. 

In [2]:
###### Test pierwszości Rabina-Millera
import random

def rabinMiller(num):
    d = num - 1    ##obliczamy wartości d i sa
    s = 0
    while d % 2 == 0:   ##usuwamy z num-1 dzielniki 2 zliczając je w s
        d = d // 2
        s += 1

    for trials in range(6):   ## wykonujemy n testów R-B 
        x = random.randrange(2, num - 1)  ##losujemy baze x
        a = pow(x, d, num)   ### pierwszy wyraz ciągu
        if (a != 1) and (a != num-1): ## jeśli a nie spełnia warunków losujemy innego świadka
            i = 1
            while (i < s) and (a != (num - 1)): #generujemy ciąg R-M
                a = (a ** 2) % num   ## obliczamy kolejne wyrazy ciągu R-M: b^d, b^2d, b^4d
                if(a==1):          ## tylko ostatni wyraz może mieć wartość 1! skończyliśmy za wcześnie
                    return False ##To nie jest liczba pierwsza lub silnie pseudopierwsza
                i+=1
            if(a!=num-1):  ##przedostatni wyraz musi mieć wartość num-1    
                return False  ##To nie jest liczba pierwsza lub silnie pseudopierwsza                
    ### jeśli wykonaliśmy n testów i żaden nie zakończył się False 
    return True

def isPrime(num):
    if (num <= 2):
        return False # oczywista oczywistość 
    #opcjonalne można sprawdzić czy małe liczby pierwsze nie są czynnikami badanej liczby   
    return rabinMiller(num)


isPrime(65537)

True

## Zadanie
1. Napisz funkcję generującą liczbę pierwszą o określonej długości w bitach. 

In [13]:
#napisz funkcję generującą liczbę pierwszą 
def generatePrime(keysize):
    #napisz swój kod tutaj
    num = random.randint(2**(keysize-1), 2**keysize-1)
    while not isPrime(num):
        num = random.randint(2**(keysize-1), 2**keysize-1)
    return num

In [23]:
generatePrime(10)

773

## Zastanów się 
1. Znajdź zestawienie liczb silnie pseudopierwszych i sprawdź działanie (pojedynczego!) testy R-M dla takich liczb
2. Ile testów R-M należy przeprowadzić aby osiągnąć bezpieczny poziom pewności testu

### 2. Obliczamy składniki kluczy 
1. Wybieramy dwie duże liczby pierwsze $p$ i $q$
2. Pierwszym składnikiem klucza jest moduł $n$ $n=p \times q$ 
3. Poszukujemy wykładnika publicznego $e$, który jest względnie pierwszy z $(p-1)\cdot (q-1)$ (czasami używane jest w miejscu pojęcie tocjentu lub funkcji Eulera: $\phi(n) = \phi(p)\cdot \phi(q) = (p − 1)·(q − 1)$)
4. Poszukujemy wykładnika prywatnego $d$, które jest odwrotnością $e\ (mod\ (p-1)\cdot (q-1))$: $de \equiv 1  (mod\ (p-1)\cdot (q-1))$
5. Kluczem publiczny jest para $(n, e)$, kluczem prywatnym jest para $(n, d)$.

## Zadanie 

1. Napisz funkcję generującą klucze RSA o ustalonym rozmiarze

In [35]:
import random, sys, os

def generateKey(keySize):
    #napisz swój kod tutaj
    p, q = generatePrime(keySize//2), generatePrime(keySize//2)
    while p==q:
        q = generatePrime(keySize//2)
    n = p*q
    t = (p-1)*(q-1)
    e = random.randint(3, t)
    while gcd(t, e)!=1:
        e = random.randint(3, t)
    d = findModInverse(e, t)
    publicKey = n,e
    privateKey = n,d
    return (publicKey, privateKey)

def makeKeyFiles(keySize):
    public, private = generateKey(keySize)
    return public, private

print('Generujemy klucze publiczny i prywatny')
makeKeyFiles(1024)

Generujemy klucze publiczny i prywatny


((102662317987717504375326616876759013822461220137602211608727229264940341388645439540041526989477422837387594949246656136319386255229192269680298214257927698811191124701618138835319235727253909539885374824428609990186645987524733813507271297480266955857453720113318160047768207886151718952865724922590878030811,
  80849615249867770937334258801663639351005854476193313258229969554420410646675568253183518981399673087675728407126984125974136045454889059386524284032532329490354427838314050205552742856755725680497192628625637161314640246547214534103415776755325271898561727541228024882190880784672808685199158111991739015411),
 (102662317987717504375326616876759013822461220137602211608727229264940341388645439540041526989477422837387594949246656136319386255229192269680298214257927698811191124701618138835319235727253909539885374824428609990186645987524733813507271297480266955857453720113318160047768207886151718952865724922590878030811,
  35561263800415219671389735044648488806083195057338886717920

## Zadanie 

Napisz funkcje implementujące szyfrowanie i deszyfrowanie RSA (tzw. podręcznikowe)

### Szyfrowanie RSA 
Operacja szyfrowania: $c=m^e (mod\ n)$

In [36]:
def encrypt(message, modulus, exp):
    # kod szyfrowania     
    message_encrypted = [pow(ord(char), exp, modulus) for char in message]
    return message_encrypted




### Deszyfrowanie RSA 
Operacja szyfrowanie $m = c^d (mod\ n)$

In [37]:
def decrypt(message_encrypted, modulus, exp):
    message_ascii = [chr(pow(char, exp, modulus)) for char in message_encrypted]
    return ('').join(message_ascii)



In [39]:
pub, priv = makeKeyFiles(1024)
msg = "Wiadomosc do przetestowania"
print("Szyfrogram:", encrypt(msg, *pub))
print("Wiadomość odszyfrowana:", decrypt(encrypt(msg, *pub), *priv))

Szyfrogram: [57265396592719721828167290669642899589162954461995712524322371625080057569820960724170710933932817187538048934766702405425588897057843397812879400980597246357006680848421595074626073931094054918322766718985018739984616865484119272416716608563304842631128174835253892646531496633839491920282944198516905306571, 70965219766525393817767849252233850425642460676135682752651735415658078221084186589401555668775451507426411681982377465844761594089025257226835448435572077798909716030041855738859503873509798678149136114100112920592409163667085181126164500761185316199643452217664645375322334292970393181485615818606285973158, 50225548252830728992768902078592556571858005241914589708073051382838698413710246806385970973064186455965343596203269082209671067208784979424309548345833265051243517712194749508885863603061111933185967673847352665795586474225109094759752631485002893470595032187235063573922038690361796018737813229908672182673, 665463095698591166376647168296798460845492932552262714914

Wiadomość odszyfrowana: Wiadomosc do przetestowania


## Zastanów się
1. Sprawdź działanie powyższej implementacji dla różnych wielkości klucza (podawane podczas generowania kluczy). 
2. Poszukaj informacji o trybie podręcznikowym RSA (*textbook RSA encryption*). Na czym polega? Jakie są jego wady i zalety? 


## Zadanie 
Zademonstruj deformowalność podręcznikowego szyfrowania RSA